Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import matplotlib.pyplot as plt
import cv2
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D,MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
#from keras.preprocessing.image import 
from keras.utils import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import pandas as pd
import tensorflow as tf
import keras
import pickle


2023-05-25 01:41:50.910006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
base_dir='/Users/dilnazahmetzan/Downloads/capstone2'
import os
for dirname, _, filenames in os.walk('/Users/dilnazahmetzan/Downloads/capstone2'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [3]:
train_list_mod=['acne','wrinkles','carcinoma','blackhead','norm']

In [4]:
def data_dictionary():
    path_train="/Users/dilnazahmetzan/Downloads/capstone2/train/"
    path_test="/Users/dilnazahmetzan/Downloads/capstone2/test/"
    list_train=train_list_mod#os.listdir(path_train)
    train_dictionary={"image_path":[],"target":[]}
    test_dictionary={"image_path":[],"target":[]}
    k=0
    for i in list_train:
        path_disease_train=path_train+i
        path_disease_test=path_test+i
        image_list_train=os.listdir(path_disease_train)
        image_list_test=os.listdir(path_disease_test)
        for j in image_list_train:
            img_path_train=path_disease_train+"/"+j
            train_dictionary["image_path"].append(img_path_train)
            train_dictionary['target'].append(k) 
        for m in image_list_test :
            img_path_test=path_disease_test+"/"+m
            test_dictionary["image_path"].append(img_path_test)
            test_dictionary['target'].append(k)
        k+=1 
    test_df=pd.DataFrame(test_dictionary)
    train_df=pd.DataFrame(train_dictionary)
        
    return  train_df,test_df

In [5]:
def load_data(input_size=(100,100)):
    images=[]
    images2=[]
    train_df,test_df=data_dictionary()
    pixels=[]
    k = 0 
    t = 0
    for i in train_df['image_path']:
        img=cv2.imread(i)
        if img is None:
           print('Wrong path:',i )
        else:
           img = cv2.resize(img, input_size)
           pixels.append(img)
        #img=cv2.resize(img,input_size)
        images.append(img)
        k=k+1
        print(k)
    y_train=np.asarray(train_df['target'])
    x_train=np.asarray(images)
    for i in test_df['image_path']:
        img=cv2.imread(i)
        if img is None:
           print('Wrong path:',i )
        else:
           img = cv2.resize(img, input_size)
           pixels.append(img)
        #img=cv2.resize(img,input_size)
        images2.append(img)
        t=t+1
        print(t)
    y_test=np.asarray(test_df['target'])
    x_test=np.asarray(images2)
    return x_train,x_test,y_train,y_test

In [6]:
x_train,x_test,y_train,y_test=load_data(input_size=(224,224))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [7]:
from keras.applications import  VGG19
from keras.models import Model
from keras.applications.vgg19 import preprocess_input

In [8]:
vgg16 = VGG19(include_top=False,weights='imagenet')

In [9]:
import joblib

In [10]:
filename="vgg16.sav"
joblib.dump(vgg16,open(filename,'wb'))

In [11]:
vgg19=joblib.load("vgg16.sav")

In [12]:
train_img=preprocess_input(x_train)
test_img=preprocess_input(x_test)

In [13]:
features_train=vgg16.predict(train_img)

21/21 [==============================] - 295s 14s/step


In [14]:
features_test=vgg16.predict(test_img)

6/6 [==============================] - 69s 11s/step


In [20]:
features_train.shape

(670, 7, 7, 512)

In [21]:
shape = x_train.shape
print(shape)

(670, 25088)


In [22]:
features_test.shape

(168, 7, 7, 512)

In [23]:
num_train=x_train.shape[0]
num_test=x_test.shape[0]
print(num_train)
print(num_test)

670
168


In [24]:
x_train=features_train.reshape(num_train,25088)
x_test=features_test.reshape(num_test,25088)
print(x_train, x_test)

[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        4.126216  0.       ]
 [0.        0.        0.        ... 0.        5.3241863 2.8400168]
 [0.        0.        0.        ... 0.        0.        0.       ]] [[0.        0.        0.        ... 0.        5.7652464 0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        2.3063517 0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


In [20]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten, Dense

model=Sequential([[ Dense(1024,activation='relu'),
                   Dense(512,activation='relu'),
                  Dense(256,activation='relu'),
                  Dense(128,activation='relu'),
                  Dense(5,activation='sigmoid'),

]])

In [25]:
from keras import layers

In [26]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal",
                          input_shape=(224,
                                       224,
                                       3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomBrightness(0.1),
    ]
)

model=keras.Sequential()
model.add(data_augmentation)
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(5,activation='relu'))

In [27]:
model = keras.Sequential()
model.add(data_augmentation)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(5, activation='relu'))

In [28]:
my_callbacks = [
    keras.callbacks.ModelCheckpoint(filepath='model.{val_accuracy:.2f}.h5')
]

In [29]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [30]:
model.fit(x_train,y_train,batch_size=32,epochs=40,validation_data=(x_test,y_test),callbacks=my_callbacks)

Epoch 1/40


ValueError: in user code:

    File "/Users/dilnazahmetzan/anaconda3/envs/capstone/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/dilnazahmetzan/anaconda3/envs/capstone/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/dilnazahmetzan/anaconda3/envs/capstone/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/dilnazahmetzan/anaconda3/envs/capstone/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/Users/dilnazahmetzan/anaconda3/envs/capstone/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Us

In [ ]:
#plt.plot(model.history.history['accuracy'],label="train_accuracy")
plt.plot(model.history.history['val_accuracy'],label="validation_accuracy")
plt.legend()
plt.show()

In [ ]:
def load_img(img_path):
    images=[]
    img=cv2.imread(img_path)
    img=cv2.resize(img,(100,100))
    images.append(img)
    x_test=np.asarray(images)
    test_img=preprocess_input(x_test)
    features_test=vgg16.predict(test_img)
    num_test=x_test.shape[0]
    f_img=features_test.reshape(num_test,4608)
    
    return f_img

In [ ]:
import pickle

In [ ]:
!mkdir -p saved_model
model.save('saved_model/skin_model')

In [ ]:
model = tf.keras.models.load_model('saved_model/skin_model')

In [ ]:
img=load_img("/content/drive/MyDrive/acne2/testing/keratosis/actinic-keratosis-face-51.jpg")

In [ ]:
img

In [ ]:
np.argmax(model.predict(img))

In [ ]:
model.save("my_h5_model.h5")


In [ ]:
import tensorflowjs

In [ ]:
!tensorflowjs_converter \
    --input_format=keras \
    model.0.h5 \
    my_tfjs_model_new